In [1]:
# Importing libaries
import pandas as pd
import datetime as dt

In [3]:
# Load your dataset here
data = pd.read_excel('1.xlsx')

In [4]:
# Convert InvoiceDate to datetime and create TotalPrice column
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data['TotalPrice'] = data['Quantity'] * data['Price']

# Drop rows with missing Customer ID
data = data.dropna(subset=['Customer ID'])

In [6]:
# Set a reference date for recency calculation
latest_date = data['InvoiceDate'].max() + dt.timedelta(days=1)

# RFM calculation
rfm = data.groupby('Customer ID').agg({
    'InvoiceDate': lambda x: (latest_date - x.max()).days,
    'Invoice': 'count',
    'TotalPrice': 'sum'
}).rename(columns={'InvoiceDate': 'Recency',
                   'Invoice': 'Frequency',
                   'TotalPrice': 'MonetaryValue'})

# Display the first few rows of the RFM DataFrame
rfm.head()

,Recency,Frequency,MonetaryValue
Customer ID,,,
12346.0,67,46,-64.68
12347.0,3,71,1323.32
12348.0,74,20,222.16
12349.0,43,107,2646.99
12351.0,11,21,300.93


In [7]:
# Assign quartile values to each parameter
rfm['R_quartile'] = pd.qcut(rfm['Recency'], 4, ['1','2','3','4'])
rfm['F_quartile'] = pd.qcut(rfm['Frequency'], 4, ['4','3','2','1'])
rfm['M_quartile'] = pd.qcut(rfm['MonetaryValue'], 4, ['4','3','2','1'])

# Combine RFM quartile scores
rfm['RFM_Score'] = rfm['R_quartile'].astype(str) + rfm['F_quartile'].astype(str) + rfm['M_quartile'].astype(str)

# Display the first few rows of the segmented RFM DataFrame
rfm.head()

,Recency,Frequency,MonetaryValue,R_quartile,F_quartile,M_quartile,RFM_Score
Customer ID,,,,,,,
12346.0,67,46,-64.68,3,2,4,324
12347.0,3,71,1323.32,1,2,2,122
12348.0,74,20,222.16,3,3,4,334
12349.0,43,107,2646.99,2,1,1,211
12351.0,11,21,300.93,1,3,3,133
